This is not important for the task. We keep it just temporary for inspiration.

In [77]:
import os
import cv2
import numpy as np

folder= r"images"
image_array=[]

for filename in os.listdir(folder):
    input_path=os.path.join(folder, filename)
    image=cv2.imread(input_path)
    if image is None:
        print("image not found")
        continue
image_array.append(np.array(image))




KeyboardInterrupt: 

Here we will process our data. The code creates new dataframes which label every image to a type (for question t01, t02 and t07) according to the format used in the CustomImageDataset class.

In [202]:
import os
import cv2
import numpy as np
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from PIL import Image


img_dir=r"images"
csv_file_number=r"gz2_filename_mapping.csv"
csv_file_quest=r"gz2_hart16.csv"

file_number=pd.read_csv(csv_file_number)
file_quest=pd.read_csv(csv_file_quest)

# We use data from 2 different files: one that maps every image to its id and one that maps every id to the questions answers
# So we match every image to the corresponding answers
# We have taken the weighted fraction for the answers to questions t01, t02 and t07
#  
file_merge = file_quest.merge(file_number, left_on='dr7objid', right_on='objid')[['objid', 'asset_id', 't01_smooth_or_features_a01_smooth_weighted_fraction', 't01_smooth_or_features_a02_features_or_disk_weighted_fraction', 't01_smooth_or_features_a03_star_or_artifact_weighted_fraction', 't02_edgeon_a04_yes_weighted_fraction', 't02_edgeon_a05_no_weighted_fraction', 't07_rounded_a16_completely_round_weighted_fraction', 't07_rounded_a17_in_between_weighted_fraction', 't07_rounded_a18_cigar_shaped_weighted_fraction']]

print(file_merge)
file_merge.to_csv('gz2_merge.csv', index=False)


                     objid  asset_id  \
0       587732591714893851     58957   
1       588009368545984617    193641   
2       587732484359913515     55934   
3       587741723357282317    158501   
4       587738410866966577    110939   
...                    ...       ...   
239690  587741490371625059    262969   
239691  587731174917669027    275883   
239692  587731512070177108    282536   
239693  587731513145688256    284172   
239694  588015507679805636    288961   

        t01_smooth_or_features_a01_smooth_weighted_fraction  \
0                                                   0.000     
1                                                   0.024     
2                                                   0.780     
3                                                   0.036     
4                                                   0.767     
...                                                   ...     
239690                                              0.823     
239691         

The following code iterates through the new dataframe and checks which images will open and which cannot open (are probably missing). In the case that the image is missing it will remove its entry from the dataframe.

In [208]:
img_dir=r"images"
csv_file=r"gz2_merge.csv"

file=pd.read_csv(csv_file)
file['asset_id'] = file['asset_id'].astype(str) + ".jpg"

for ind in file.index:
    input_path=os.path.join(img_dir, file['asset_id'][ind])
    photo=cv2.imread(input_path)
    if photo is None:
        file.drop(ind, inplace=True)

file.to_csv('existing_images.csv', index= False)
        

We atributed a type to images. In order for an image to be labeled into a type, we have put the condition that the corresponding weighted fraction should be greater than 0.6 ( basically more than 60% from the people should have labeled it this way). We have created 3 different dataframes corresponding to the classification in raport to questions t01, t02 and t07.

In [211]:
file.loc [file['t01_smooth_or_features_a01_smooth_weighted_fraction']>0.6, 't01_type'] = 0
file.loc [file['t01_smooth_or_features_a02_features_or_disk_weighted_fraction']>0.6, 't01_type'] = 1

# Here we have made an exception as the class is extremely sparse so we have labeled it for fractions that are at least 0.5

file.loc [file['t01_smooth_or_features_a03_star_or_artifact_weighted_fraction']>=0.5, 't01_type'] = 2

file.dropna(inplace=True)
file.reset_index(drop=True, inplace=True)
t01_file=file[['asset_id','t01_type']].copy()

file_modif2=file.copy()
file_modif7=file.copy()

file_modif2.loc [file['t02_edgeon_a04_yes_weighted_fraction']>0.6, 't02_type'] = 0
file_modif2.loc [file['t02_edgeon_a05_no_weighted_fraction']>0.6, 't02_type'] = 1

file_modif2.dropna(inplace=True)
file_modif2.reset_index(drop=True, inplace=True)
t02_file=file_modif2[['asset_id','t02_type']].copy()

file_modif7.loc [file['t07_rounded_a16_completely_round_weighted_fraction']>0.6, 't07_type'] = 0
file_modif7.loc [file['t07_rounded_a17_in_between_weighted_fraction']>0.6, 't07_type'] = 1
file_modif7.loc [file['t07_rounded_a18_cigar_shaped_weighted_fraction']>0.6, 't07_type'] = 2

file_modif7.dropna(inplace=True)
file_modif7.reset_index(drop=True, inplace=True)
t07_file=file_modif7[['asset_id','t07_type']].copy()

print(t01_file)
print(t02_file)
print(t07_file)
#t01_file.to_csv('unbalanced_t01.csv',index=False)


          asset_id  t01_type
0        58957.jpg       1.0
1       193641.jpg       1.0
2        55934.jpg       0.0
3       158501.jpg       1.0
4       110939.jpg       0.0
...            ...       ...
209734   30593.jpg       0.0
209735   74398.jpg       0.0
209736  151540.jpg       0.0
209737  259387.jpg       0.0
209738  262969.jpg       0.0

[209739 rows x 2 columns]
          asset_id  t02_type
0        58957.jpg       1.0
1       193641.jpg       1.0
2        55934.jpg       1.0
3       158501.jpg       1.0
4       110939.jpg       1.0
...            ...       ...
197172  264076.jpg       0.0
197173   30593.jpg       1.0
197174  151540.jpg       0.0
197175  259387.jpg       1.0
197176  262969.jpg       0.0

[197177 rows x 2 columns]
          asset_id  t07_type
0       193641.jpg       0.0
1        55934.jpg       1.0
2       158501.jpg       0.0
3       249897.jpg       1.0
4        71801.jpg       1.0
...            ...       ...
177746  264076.jpg       2.0
177747   30593.jpg

We will now identify how balanced our dataset is by counting how many images correspond to each type.

In [216]:
print( f"t01 type 0 images: { t01_file['t01_type'].value_counts()[0] }")
print( f"t01 type 1 images: { t01_file['t01_type'].value_counts()[1] }")
print( f"t01 type 2 images: { t01_file['t01_type'].value_counts()[2] }")

print( f"t02 type 0 images: { t02_file['t02_type'].value_counts()[0] }")
print( f"t02 type 1 images: { t02_file['t02_type'].value_counts()[1] }")

print( f"t07 type 0 images: { t07_file['t07_type'].value_counts()[0] }")
print( f"t07 type 1 images: { t07_file['t07_type'].value_counts()[1] }")
print( f"t07 type 2 images: { t07_file['t07_type'].value_counts()[2] }")

t01 type 0 images: 159578
t01 type 1 images: 50033
t01 type 2 images: 128
t02 type 0 images: 21860
t02 type 1 images: 175317
t07 type 0 images: 59087
t07 type 1 images: 93277
t07 type 2 images: 25387


As we can see t01 type 2 is very sparse so we will habe to perform some data augmentation.